## Image Generator
#### It systematically tries every combination of Color Palette × Pattern × Motif × Theme × Finish.

### How It Works
- Inputs: Five lists of design attributes (palettes, patterns, motifs, themes, finishes).
- Combinations: 4 × 5 × 6 × 4 × 4 = `1920` total prompts.

### Cost Considerations

- Estimated cost per `1024×1024` image: `$0.04`
- 1920 images total: `~$76.80` (rounded)
- Cost depends on resolution and output token count.
- Running the full set is feasible, but expensive

### Time & Rate Limits
- Default rate limit: ~50 requests per minute.
- At 50 RPM, 1920 images take ~40 minutes.
- script includes:
    - Sliding-window rate limiter
    - Exponential backoff with jitter for 429 / 5xx errors
    - Checkpointing (skips files already generated)

In [92]:
import os
import time
import base64
import itertools
import random
import traceback
from collections import deque
from datetime import datetime
from typing import Iterable, Tuple

from openai import OpenAI
from openai import APIError, APIConnectionError, RateLimitError, InternalServerError

## Just Switch the Model here

In [16]:
MODEL_PROVIDER = "openai"
# MODEL_PROVIDER = "gemini"

### SET the API key for the model you want to use

In [ ]:
# ========== CONFIG ==========
OPENAI_API_KEY =""
# OPENAI_API_KEY = ""
OPENAI_MODEL = "dall-e-3"
OPENAI_SIZE = "1024x1024"

GEMINI_API_KEY= " "
GEMINI_MODEL = "gemini-2.0-flash-image-preview"  # image-capable Gemini model
GEMINI_SIZE = "1024x1024"

In [68]:
# Rate limit window (tune to your plan; example: 50 requests/min)
REQUESTS_PER_MINUTE = 50
# Safety cap for a single run; set None to run all
MAX_JOBS = None  # e.g., 50 for quick smoke tests

# Retry policy
MAX_RETRIES = 3
BASE_BACKOFF_SECONDS = 2.0  # exponential backoff base
BASE_BACKOFF = 2.0
BACKOFF_JITTER = (0.0, 0.75)  # add random jitter to spread bursts

# Output
OUT_DIR = "outputs"
os.makedirs(OUT_DIR, exist_ok=True)

In [32]:
# ----- Import + normalize exception types so both providers hit the same handlers -----
# Defaults in case imports fail (lets the handlers still work)
class _FallbackRateLimitError(Exception): ...


class _FallbackAPIError(Exception): ...


class _FallbackAPIConnectionError(Exception): ...


class _FallbackInternalServerError(Exception): ...


RateLimitError = _FallbackRateLimitError
APIError = _FallbackAPIError
APIConnectionError = _FallbackAPIConnectionError
InternalServerError = _FallbackInternalServerError

### Errors and Exceptions

In [33]:
# Try to import OpenAI exception classes
try:
    from openai import RateLimitError as _OpenAIRateLimitError
    from openai import APIError as _OpenAIAPIError
    from openai import APIConnectionError as _OpenAIAPIConnectionError
    from openai import InternalServerError as _OpenAIInternalServerError

    RateLimitError = _OpenAIRateLimitError
    APIError = _OpenAIAPIError
    APIConnectionError = _OpenAIAPIConnectionError
    InternalServerError = _OpenAIInternalServerError
except Exception:
    pass

# Map Gemini (google) exceptions into our normalized set
_GEMINI_TRANSIENT_CODES = {500, 502, 503, 504}
try:
    import google.generativeai as genai

    # google.api_core exceptions:
    from google.api_core.exceptions import (
        ResourceExhausted,
        InternalServerError as GInternalServerError,
    )
    from google.api_core.exceptions import ServiceUnavailable, DeadlineExceeded
    from google.api_core.exceptions import GoogleAPICallError

    _HAVE_GEMINI_EXC = True
except Exception:
    _HAVE_GEMINI_EXC = False

In [76]:
def _raise_normalized_from_gemini(e):
    """
    Convert Gemini exceptions to the same types we use for OpenAI so the shared
    retry/limit block works IDENTICALLY.
    """
    msg = str(e)
    code = getattr(e, "code", None) or getattr(e, "status", None)
    if _HAVE_GEMINI_EXC and isinstance(e, ResourceExhausted):
        raise RateLimitError(msg)
    if isinstance(e, (GInternalServerError, ServiceUnavailable, DeadlineExceeded)):
        # transient network/server
        err = APIError(msg)
        setattr(err, "status", getattr(e, "code", None))
        raise err
    if isinstance(e, GoogleAPICallError) and code in _GEMINI_TRANSIENT_CODES:
        err = APIError(msg)
        setattr(err, "status", code)
        raise err
    # Unknown -> bubble up; main handler will catch as Exception
    raise

In [77]:
# ----- Rate Limiter (sliding window) -----
# --- Rate Limiter ---
class SlidingWindowRateLimiter:
    """Simple sliding-window rate limiter to enforce requests-per-minute caps."""
    def __init__(self, rpm: int):
        self.capacity = max(1, rpm)
        self.window = 60.0
        self.events = deque()

    def wait_for_slot(self):
        now = time.time()
        while self.events and (now - self.events[0]) > self.window:
            self.events.popleft()
        if len(self.events) < self.capacity:
            self.events.append(now)
            return
        sleep_for = (self.events[0] + self.window) - now + 0.01
        time.sleep(sleep_for)
        self.events.append(time.time())


rate_limiter = SlidingWindowRateLimiter(REQUESTS_PER_MINUTE)

## Testing with openai

In [69]:
# color_palettes_t = [
#     "pastel pinks",
#     "jewel tones",
# ]

# patterns_t = [
#     "stripes",
#     "chevrons",
# ]

# motifs_t = [
#     "pumpkins",
# ]

# themes_t = [
#     "whimsical gothic",
# ]

# finishes_t = [
#     "matte",
# ]

In [70]:
color_palettes = [
    "pastel pinks",
    "jewel tones",
    "metallic gold & black",
    "earthy autumn shades",
]

patterns = [
    "stripes",
    "chevrons",
    "damask",
    "watercolor wash",
    "geometric lattice",
]

motifs = [
    "pumpkins",
    "bats",
    "florals",
    "stars",
    "waves",
    "shells",
]

themes = [
    "whimsical gothic",
    "festive holiday sparkle",
    "coastal summer",
    "rustic harvest",
]

finishes = [
    "matte",
    "foil stamping",
    "embossed texture",
    "glossy lacquer",
]

PROMPT_TEMPLATE = (
    "Design a premium, print-ready paper plate featuring {color_palette}, "
    "with {pattern} as the base, highlighted by {motif} in a {theme} style. "
    "Incorporate finishing details like {finish} for a polished, commercial look. "
    "The design should balance aesthetics with festive usability and align with U.S. "
    "retail holiday/gifting trends."
)

In [78]:
def safe_filename(*parts: str) -> str:
    """Generate a safe, filesystem-friendly filename from text parts."""
    name = "__".join(parts)
    # sanitize
    name = name.replace("&", "and")
    for ch in r' <>:"/\|?*':
        name = name.replace(ch, "_")
    return name[:200]  # guard against super long names


def combos() -> Iterable[Tuple[str, str, str, str, str]]:
    """Yield all combinations of palette, pattern, motif, theme, and finish (optionally limited)."""
    iterable = itertools.product(color_palettes, patterns, motifs, themes, finishes)
    if MAX_JOBS is not None:
        # take just the first N combos for testing
        iterable = itertools.islice(iterable, MAX_JOBS)
    return iterable

## testing with small combinations
# def combos_test() -> Iterable[Tuple[str, str, str, str, str]]:
#     iterable_t = itertools.product(color_palettes_t, patterns_t, motifs_t, themes_t, finishes_t)
#     if MAX_JOBS is not None:
#         # take just the first N combos for testing
#         iterable = itertools.islice(iterable_t, MAX_JOBS)
#     return iterable_t

In [79]:
def _call_openai(prompt: str) -> bytes:
    """Call OpenAI Image API with a prompt and return raw PNG bytes."""
    from openai import OpenAI

    client = OpenAI(api_key=OPENAI_API_KEY)
    result = client.images.generate(
        model=OPENAI_MODEL,
        prompt=prompt,
        size=OPENAI_SIZE,
        response_format="b64_json",
    )
    b64 = result.data[0].b64_json
    return base64.b64decode(b64)


def _call_gemini(prompt: str) -> bytes:
    """Call Gemini Image API with a prompt and return raw image bytes."""
    if not _HAVE_GEMINI_EXC:
        import google.generativeai as genai  # fallback import
    genai.configure(api_key=GEMINI_API_KEY)
    model = genai.GenerativeModel(GEMINI_MODEL)
    # Some SDK versions use generate_images; if you use a different method, adapt here.
    resp = model.generate_images(prompt=prompt, size=GEMINI_SIZE)
    # Depending on SDK version, image bytes accessor can differ:
    # Try common attribute names in order.
    img = resp.images[0]
    for attr in ("_image_bytes", "image_bytes", "bytes", "data"):
        if hasattr(img, attr):
            return getattr(img, attr)
    # If SDK returns base64 string:
    if hasattr(img, "b64_json"):
        return base64.b64decode(img.b64_json)
    raise RuntimeError("Unable to extract image bytes from Gemini response")

In [80]:
def generate_with_retries(call_fn, filename_stem: str) -> bytes:
    """Wrap image API call with rate limiting, retries, and exponential backoff."""
    attempt = 0
    while True:
        rate_limiter.wait_for_slot()
        try:
            return call_fn()

        except RateLimitError as e:
            # Back off on 429
            attempt += 1
            if attempt > MAX_RETRIES:
                print(
                    f"❌ Rate limited (exhausted retries) for: {filename_stem} -> {e}"
                )
                return b""
            backoff = BASE_BACKOFF_SECONDS * (2 ** (attempt - 1)) + random.uniform(
                *BACKOFF_JITTER
            )
            print(
                f"⏳ Rate limited (attempt {attempt}/{MAX_RETRIES}). Sleeping {backoff:.2f}s"
            )
            time.sleep(backoff)

        except (APIConnectionError, InternalServerError, APIError) as e:
            # Transient network or 5xx – retry with backoff
            status = getattr(e, "status", None)
            is_transient = (status is None) or (500 <= status < 600)
            attempt += 1
            if (not is_transient) and (attempt > 1):
                # Non-transient (e.g., 400) -> don't keep retrying endlessly
                print(f"❌ Non-retryable error for: {filename_stem} -> {e}")
                return b""

            if attempt > MAX_RETRIES:
                print(
                    f"❌ API/Network error (exhausted retries) for: {filename_stem} -> {e}"
                )
                return b""

            backoff = BASE_BACKOFF_SECONDS * (2 ** (attempt - 1)) + random.uniform(
                *BACKOFF_JITTER
            )
            print(
                f"⏳ Transient error (attempt {attempt}/{MAX_RETRIES}). Sleeping {backoff:.2f}s"
            )
            time.sleep(backoff)

        except Exception as e:
            # Unknown error; log and continue
            traceback.print_exc()
            print(f"❌ Unexpected error for: {filename_stem} -> {e}")
            return b""

In [81]:
def generate_openai(prompt: str, filename_stem: str) -> bytes:
    """Generate an image with OpenAI using shared retry logic and return PNG bytes."""
    def _fn():
        return _call_openai(prompt)

    return generate_with_retries(_fn, filename_stem)


def generate_gemini(prompt: str, filename_stem: str) -> bytes:
    """Generate an image with Gemini using shared retry logic and return PNG bytes."""
    def _fn():
        try:
            return _call_gemini(prompt)
        except Exception as e:
            traceback.print_exc()
            # Map Gemini exceptions into our normalized set *before* the shared handler sees them
            try:
                _raise_normalized_from_gemini(e)
            except Exception as inner:  # re-throw mapped or original
                raise inner

    return generate_with_retries(_fn, filename_stem)

In [ ]:
def info_for_user():
    all_combos = list(combos())
    total_jobs = len(all_combos)
    minutes_est = total_jobs / REQUESTS_PER_MINUTE
    est_time = f"~{minutes_est:.1f} minutes"
    est_cost = total_jobs * 0.04 if MODEL_PROVIDER == "openai" else "N/A"

    print("=======================================")
    print(f"Provider        : {MODEL_PROVIDER}")
    print(f"Total jobs      : {total_jobs}")
    print(f"Rate limit      : {REQUESTS_PER_MINUTE} req/min")
    print(f"Estimated time  : {est_time}")
    if MODEL_PROVIDER == "openai":
        print(f"Estimated cost  : ${est_cost:.2f} (at $0.04 per image)")
    print("=======================================")

    print('Enter y for yes and n for No in the input box')

    proceed = input("Proceed with generation? (y/n): ").strip().lower()
        
    return proceed == 'y'

In [94]:
def main():
    """Main loop: build prompts, generate images with selected provider, and save outputs."""
    proceed = info_for_user()
    if not proceed:
        print("❌ Aborted by user.")
        return
    total = 0
    for color, pattern, motif, theme, finish in combos():
        stem = safe_filename(color, pattern, motif, theme, finish)
        out_path = os.path.join(OUT_DIR, f"{stem}.png")
        if os.path.exists(out_path):
            print(f"⏭️  Exists, skipping: {stem}")
            continue

        prompt = PROMPT_TEMPLATE.format(
            color_palette=color,
            pattern=pattern,
            motif=motif,
            theme=theme,
            finish=finish,
        )
        print(f"→ Generating via {MODEL_PROVIDER}: {stem}")

        if MODEL_PROVIDER == "openai":
            img = generate_openai(prompt, stem)
        elif MODEL_PROVIDER == "gemini":
            img = generate_gemini(prompt, stem)
        else:
            raise ValueError(f"Unknown provider: {MODEL_PROVIDER}")

        if not img:
            # generation failed but logged already
            continue

        with open(out_path, "wb") as f:
            f.write(img)

        meta = os.path.join(OUT_DIR, f"{stem}.meta.txt")
        with open(meta, "w", encoding="utf-8") as f:
            f.write(f"provider={MODEL_PROVIDER}\n")
            f.write(
                f"model={OPENAI_MODEL if MODEL_PROVIDER=='openai' else GEMINI_MODEL}\n"
            )
            f.write(
                f"size={OPENAI_SIZE if MODEL_PROVIDER=='openai' else GEMINI_SIZE}\n"
            )
            f.write(f"prompt={prompt}\n")
            f.write(f"time={datetime.utcnow().isoformat()}Z\n")

        print(f"✅ Saved: {out_path}")
        total += 1

    print(f"\nDone. Generated {total} images to {OUT_DIR}")

## preview prompts

In [102]:
import textwrap


def preview_prompts_filtered(
    color: str = None,
    pattern: str = None,
    motif: str = None,
    theme: str = None,
    finish: str = None,
    limit: int = 20,
    width: int = 100,
):
    """
    Generate and print prompts with optional filters on any category.
    Args:
        color, pattern, motif, theme, finish: Filter values (exact match).
        limit: Max number of prompts to show. Default 20. None = all.
        width: Wrap width for readability.
    """
    all_combos = list(combos())
    filtered = []

    for c, p, m, t, f in all_combos:
        if color and c != color:
            continue
        if pattern and p != pattern:
            continue
        if motif and m != motif:
            continue
        if theme and t != theme:
            continue
        if finish and f != finish:
            continue
        filtered.append((c, p, m, t, f))

    total = len(filtered)
    print("=======================================")
    print(f"Provider configured : {MODEL_PROVIDER}")
    print(f"Total matches       : {total}")
    print("=======================================")

    shown = 0
    for c, p, m, t, f in filtered:
        prompt = PROMPT_TEMPLATE.format(
            color_palette=c,
            pattern=p,
            motif=m,
            theme=t,
            finish=f,
        )
        print(f"\n[{shown+1}]")
        print(textwrap.fill(prompt, width=width))
        print("-" * width)

        shown += 1
        if limit is not None and shown >= limit:
            break

    if limit is not None and total > limit:
        print(f"... ({total - limit} more prompts not shown)")

In [104]:
# ==================================
# function preview_prompts_filtered(limit to see)
# Show only prompts where color is "jewel tones"
# other options ------
# preview_prompts_filtered(motif="pumpkins", limit=3)
# ==================================
preview_prompts_filtered(color="jewel tones", limit=2)

Provider configured : openai
Total matches       : 480

[1]
Design a premium, print-ready paper plate featuring jewel tones, with stripes as the base,
highlighted by pumpkins in a whimsical gothic style. Incorporate finishing details like matte for a
polished, commercial look. The design should balance aesthetics with festive usability and align
with U.S. retail holiday/gifting trends.
----------------------------------------------------------------------------------------------------

[2]
Design a premium, print-ready paper plate featuring jewel tones, with stripes as the base,
highlighted by pumpkins in a whimsical gothic style. Incorporate finishing details like foil
stamping for a polished, commercial look. The design should balance aesthetics with festive
usability and align with U.S. retail holiday/gifting trends.
----------------------------------------------------------------------------------------------------
... (478 more prompts not shown)


In [105]:
preview_prompts_filtered(color="jewel tones", motif="florals")

Provider configured : openai
Total matches       : 80

[1]
Design a premium, print-ready paper plate featuring jewel tones, with stripes as the base,
highlighted by florals in a whimsical gothic style. Incorporate finishing details like matte for a
polished, commercial look. The design should balance aesthetics with festive usability and align
with U.S. retail holiday/gifting trends.
----------------------------------------------------------------------------------------------------

[2]
Design a premium, print-ready paper plate featuring jewel tones, with stripes as the base,
highlighted by florals in a whimsical gothic style. Incorporate finishing details like foil stamping
for a polished, commercial look. The design should balance aesthetics with festive usability and
align with U.S. retail holiday/gifting trends.
----------------------------------------------------------------------------------------------------

[3]
Design a premium, print-ready paper plate featuring jewel tones, 

In [107]:
# other options example
# preview_prompts_filtered() #all at once
# preview_prompts_filtered(motif="pumpkins", limit=5)
#preview_prompts_filtered(color="earthy autumn shades", pattern="damask", limit=2)

## Run this method to start generating

In [95]:
if __name__ == "__main__":
    # Minimal env checks
    if MODEL_PROVIDER == "openai" and not OPENAI_API_KEY:
        raise SystemExit("Please set OPENAI_API_KEY.")
    if MODEL_PROVIDER == "gemini" and not GEMINI_API_KEY:
        raise SystemExit("Please set GEMINI_API_KEY.")
    main()

Provider        : openai
Total jobs      : 1920
Rate limit      : 50 req/min
Estimated time  : ~38.4 minutes
Estimated cost  : $76.80 (at $0.04 per image)
Enter y for yes and n for No in the input box yellow
→ Generating via openai: pastel_pinks__stripes__pumpkins__whimsical_gothic__matte


KeyboardInterrupt: 

In [1]:
dicts = {
    "color_palette": "Pastel Pinks",
    "pattern": "Chevrons",
    "motif": "Default",
    "style": "Festive Holiday Sparkle",
    "finish": "Matte",
}


def any_default(d: dict) -> bool:
    return any(isinstance(v, str) and v.lower() == "default" for v in d.values())

print(any_default(dicts))

True


In [4]:
result = {
    "status": True,
    "type": "success",
    "images": {
        "org": "original",
        "low": "low",
        "medium": "med",
        "high": "high",
    },
}

response = result.get("images")['org']
print(response)

original


In [7]:
res = {
    "color_palette": "pastel pinks",
    "pattern": "damask",
    "motif": "pumpkins",
    "style": "festive holiday sparkle",
    "finish": "foil stamping",
    "rationale": "The delicate pastel pinks soften the traditionally ornate damask pattern, lending it a modern, inviting feel. Foil-stamped pumpkins introduce a touch of festive sparkle that's both elegant and playful, perfectly balancing seasonal charm with refined sophistication for a uniquely celebratory ambiance.",
}

print(res.get("color_palette"))

pastel pinks
